# Erosion rates from CRN concentrations

Notebook last updated by Simon M Mudd 19/12/2023

In this example we will use CAIRN (https://doi.org/10.5194/esurf-4-655-2016) to calculate erosion rates based on the concentration of $^{10}$Be in detrital sediments.

This method requires some input files that we will get from example data.

## If you are on colab

**If you are in the `docker_lsdtt_pytools` docker container, you do not need to do any of this. 
The following is for executing this code in the google colab environment only.**

If you are in the docker container you can skip to the **Download some data** section. 

First we install `lsdtopotools`. The first line downloads the package and the second installs it. The `/dev/null` stuff is just to stop the notebook printing a bunch of text to screen.  

In [ ]:
!wget https://pkgs.geos.ed.ac.uk/geos-jammy/pool/world/l/lsdtopotools2/lsdtopotools2_0.9-1geos~22.04.1_amd64.deb  &> /dev/null
!apt install ./lsdtopotools2_0.9-1geos~22.04.1_amd64.deb  &> /dev/null

The next line tests to see if it worked. If you get some output asking for a parameter file then `lsdtopotools` is installed. This notebook was tested on version 0.9.

In [ ]:
!lsdtt-basic-metrics -v

Now we install `lsdviztools`:

In [ ]:
!pip install lsdviztools  &> /dev/null

**If you are in the `docker_lsdtt_pytools` docker container, you do not need to do any of this. 
The following is for executing this code in the google colab environment only.**

If you are in the docker container you can skip to the **First get data** section. 

## Get the example data

We need to get the example files. There are 7 files

* Two files that have atmopheric data that are used to caculate cosmogenic production rates
* The DEM (a `bil` and a `hdr` file)
* Three files that have parameters for the cosmogenic analyis. 

The first two files are the same for any cosmogenic analysis. 
If you are using `lsdtopotools` you should be quite familiar with DEMs and topogaphic data. 
So after we download the files we will look in detail at the parameter files

In [ ]:
import urllib.request
urllib.request.urlretrieve("https://github.com/LSDtopotools/ExampleTopoDatasets/raw/master/CRNData/NCEP2.bin", "NCEP2.bin")
urllib.request.urlretrieve("https://github.com/LSDtopotools/ExampleTopoDatasets/raw/master/CRNData/NCEP_hgt.bin", "NCEP_hgt.bin")
urllib.request.urlretrieve("https://github.com/LSDtopotools/ExampleTopoDatasets/raw/master/CRNData/SanBern.bil", "SanBern.bil")
urllib.request.urlretrieve("https://github.com/LSDtopotools/ExampleTopoDatasets/raw/master/CRNData/SanBern.hdr", "SanBern.hdr")
urllib.request.urlretrieve("https://github.com/LSDtopotools/ExampleTopoDatasets/raw/master/CRNData/SanBern_SS_CRNData.csv", "SanBern_SS_CRNData.csv")
urllib.request.urlretrieve("https://github.com/LSDtopotools/ExampleTopoDatasets/raw/master/CRNData/SanBern_SS_CRNRasters.csv", "SanBern_SS_CRNRasters.csv")
urllib.request.urlretrieve("https://github.com/LSDtopotools/ExampleTopoDatasets/raw/master/CRNData/SanBern_SS.CRNParam", "SanBern_SS.CRNParam")

We can just look at the files to see if they are all here:

In [ ]:
!ls SanBern*

## CAIRN parameter files

Right, lets have a look at carin parameter files. We will use pandas to look at their contents.

Any CAIRN run needs three files. Their filename will contain the prefix of the DEM (here `SanBern`) and then the three file extensions:

* `_CRNData.csv`: The data containing information about the data from the samples.
* `_CRNRasters.csv`: This contains data about the underlying rasters
* `.CRNParam`: This has other parameters about the analysis. 

In [ ]:
!ls SanBern*

In [ ]:
import pandas as pd
CRNData_df = pd.read_csv("SanBern_SS_CRNData.csv")
CRNData_df.head()

All the columns in this file are required. The `sample_name`, `sample_latitude`, and `sample_longitude` should be self explanatory, although `sample_name` is always read as a string (so you can put any combinations of letters and numbers here, but please no spaces!!), and the latitude and longitude are in decimal degrees. The `nuclide` can be `Be10`, `Al26`, `C14`, or `Ne21`. The `concentration` and `AMS_uncertainty` are both in atoms/g. The final column, `standardisation`, must come from a predifined list.

The standardisations are:
* For 10Be: 

`07KNSTD`,`KNSTD`,`NIST_Certified`,`LLNL31000`,`LLNL10000`,`LLNL3000`,`LLNL1000`,`LLNL300`,`NIST_30000`,`NIST_30200`,`NIST_30300`,`NIST_30600`,`NIST_27900`,`S555`,`S2007`,`BEST433`,`BEST433N`,`S555N`,`S2007N`
* For 26Al:

`KNSTD`,`ZAL94`,`SMAL11`,`0`,`ZAL94N`,`ASTER`,`Z92-0222`

These standards are reported by the lab that processed your data. If the standard isn't recognised then the the KN standard will be applied (which just scales the measured concentration by 1). 

In [ ]:
CRNData_df = pd.read_csv("SanBern.CRNParam")
CRNData_df.head()

## Running an analysis

First we need to import some modules from `lsdviztools`:

In [ ]:
import lsdviztools.lsdbasemaptools as bmt
from lsdviztools.lsdplottingtools import lsdmap_gdalio as gio
import lsdviztools.lsdmapwrappers as lsdmw

Lets get a hillshade an look at some of the points

In [ ]:
# IMPORTANT all the parameter values must be passed as strings. 
# So even if the parameter is a number it always needs to be in quotations
lsdtt_parameters = {"write_hillshade" : "true"}
r_prefix = "SanBern"
w_prefix = "SanBern"
lsdtt_drive = lsdmw.lsdtt_driver(read_prefix = r_prefix,
                                 write_prefix= w_prefix,
                                 read_path = "./",
                                 write_path = "./",
                                 parameter_dictionary=lsdtt_parameters)
lsdtt_drive.print_parameters()
lsdtt_drive.run_lsdtt_command_line_tool()

In [ ]:
%%capture 
DataDirectory = "./"

# Use the capture comment to get rid of all the text
# But it will also not display an inline image
# So you need to call the image from the next line of code           
Base_file = r_prefix
this_img = lsdmw.SimpleHillshade(DataDirectory,Base_file,cmap="gist_earth", 
                                 save_fig=True, size_format="geomorphology",dpi=600)


In [ ]:
from IPython.display import display, Image
display(Image(filename=this_img, width=800))

We can also plot the points. This is a little annoying since the headers in the data file do not have `latitude` and `longitude`, which are required. A little bit of `pandas` will sort that out. 

In [ ]:
list(CRNData_df)

In [ ]:
import pandas as pd
CRNData_df = pd.read_csv("SanBern_SS_CRNData.csv")
CRNData_df["latitude"] =  CRNData_df[" sample_latitude"]
CRNData_df["longitude"] =  CRNData_df[" sample_longitude"]
CRNData_df.to_csv("SanBern_SS_CRNData_latlong.csv")

In [ ]:
### Plot points the hillshade
import lsdviztools.lsdmapwrappers as lsdmw
%matplotlib inline
Base_file = "SanBern"
DataDirectory = "./"
this_img = lsdmw.PrintPointsOverHillshade(DataDirectory,Base_file, column_for_plotting = "concentration", 
                                          points_fname = "SanBern_SS_CRNData_latlong.csv", 
                                          scale_points = False, manual_size =20,
                                          cmap="jet", save_fig=False, size_format="geomorphology")

## Check the cosmo data

In this next step we check to see if the cosmo data is in the right place

In [ ]:
# IMPORTANT all the parameter values must be passed as strings. 
# So even if the parameter is a number it always needs to be in quotations
lsdtt_parameters = {"cosmo_parameter_prefix" : "SanBern",
                    "check_cosmo_basins" : "true"}
r_prefix = "SanBern"
w_prefix = "SanBern"
lsdtt_drive = lsdmw.lsdtt_driver(command_line_tool = "lsdtt-cosmo-tool",
                                 read_prefix = r_prefix,
                                 write_prefix= w_prefix,
                                 read_path = "./",
                                 write_path = "./",
                                 parameter_dictionary=lsdtt_parameters)
lsdtt_drive.print_parameters()
lsdtt_drive.run_lsdtt_command_line_tool()

This gets the basins, which we can plot.

In [ ]:
%%capture
DataDirectory = "./"
Base_file = "SanBern"
basins_img = lsdmw.PrintBasins_Complex(DataDirectory,Base_file,cmap="gist_earth", 
                             size_format="geomorphology",dpi=600, save_fig = True)

In [ ]:
print(basins_img)
from IPython.display import display, Image
display(Image(filename=basins_img, width=800))

Now we are going to extract the erosion rates. This takes a little while since it automatically calculates topographic shielding, which is the most computationally expensive step. 

There is a quite good argument that topographic shielding is not required, but I've not implemented an automatic skip of this step yet. 

In [ ]:
# IMPORTANT all the parameter values must be passed as strings. 
# So even if the parameter is a number it always needs to be in quotations
lsdtt_parameters = {"print_production_raster" : "true",
                    "cosmo_parameter_prefix" : "SanBern",
                    "calculate_erosion_rates" : "true"}
r_prefix = "SanBern"
w_prefix = "SanBern"
lsdtt_drive = lsdmw.lsdtt_driver(command_line_tool = "lsdtt-cosmo-tool",
                                 read_prefix = r_prefix,
                                 write_prefix= w_prefix,
                                 read_path = "./",
                                 write_path = "./",
                                 parameter_dictionary=lsdtt_parameters)
lsdtt_drive.print_parameters()
lsdtt_drive.run_lsdtt_command_line_tool()

The results are in a file with the extension `_CRNResults.csv`

This file has 3 rows of text at the beginning so to load it you need to use the `skiprows` keyword. 

In [ ]:
CRNResults_df = pd.read_csv("SanBern_CRNResults.csv",skiprows=3)
CRNResults_df.head()

If we want to plot this we need to get rid of the header

In [ ]:
CRNResults_df.to_csv("SanBern_CRNResults_dropheader.csv")

In [ ]:
### Plot points the hillshade
import lsdviztools.lsdmapwrappers as lsdmw
%matplotlib inline
Base_file = "SanBern"
DataDirectory = "./"
this_img = lsdmw.PrintPointsOverHillshade(DataDirectory,Base_file, column_for_plotting = "erate_mmperkyr_rho2650", 
                                          points_fname = "SanBern_CRNResults_dropheader.csv", 
                                          scale_points = False, manual_size =30,
                                          cmap="jet", save_fig=False, size_format="geomorphology")

## Get points using the updated CRN erosion rate routine

We have developed an erosion rate calculator with a simpler interface. 

You just need a file with latitude, longitude and concentrations. 

In [ ]:
# IMPORTANT all the parameter values must be passed as strings. 
# So even if the parameter is a number it always needs to be in quotations
lsdtt_parameters = {"points_filename" : "SanBern_SS_CRNData_latlong.csv",
                    "nuclide_for_prediction" : "Be10",
                    "concentration_column_name" : "concentration",
                    "calculate_erosion_rates_new" : "true"}
r_prefix = "SanBern"
w_prefix = "SanBern"
lsdtt_drive = lsdmw.lsdtt_driver(command_line_tool = "lsdtt-cosmo-tool",
                                 read_prefix = r_prefix,
                                 write_prefix= w_prefix,
                                 read_path = "./",
                                 write_path = "./",
                                 parameter_dictionary=lsdtt_parameters)
lsdtt_drive.print_parameters()
lsdtt_drive.run_lsdtt_command_line_tool()

In [ ]:
!ls *.csv

In [ ]:
New_results_df = pd.read_csv("SanBern_CN_ERATE.csv")
New_results_df.head()

In [ ]:
Lets see how close this came to the CAIRN method. This doesn't have the topographic shielding component, mainly be=cause of theoretical work by DiBiase that showed topographic shielding is offset by greater surface exposure in sloping terrain (https://esurf.copernicus.org/articles/6/923/2018/esurf-6-923-2018.html) 